# Turbulence Simulation with Julia: Kolmogorov Flow

**Author**: Mingyu Jeon   
**Date**: 2025-08-07

---

Kolmogorov flow is a shear flow with spatially periodic forcing {cite:p}`Chatterjee2020`. It is a well-known test-bed for simulating turbulence {cite:p}`fylladitakis2018kolmogorov`.

The Navier-Stokes equations for an incompressible fluid are

$$
\frac{\partial \mathbf{u}}{\partial t} + \mathbf{u} \cdot \nabla \mathbf{u} = - \frac{1}{\rho}\nabla p + \nu \nabla^2 \mathbf{u} + \mathbf{f}
$$

$$
\nabla\cdot \mathbf{u} = 0
$$

where  $\rho$ is the density, $\mathbf{u}$ is the velocity, $p$ is the pressure, $\nu$ is the kinematic viscosity, and $\mathbf{f}$ (body force / density) is the "forcing" (or acceleration).

We assume that density $\rho$ is constant.

When we consider a 2D Kolmogorov flow in a periodic and rectangular domain in the xy-plane, the "forcing" $\mathbf{f}$ is given by

$$
\mathbf{f} = c \sin(k y) \hat{x}
$$

where $c$ and $k$ are constants.

In this post, we will simulate the 2D Kolmogorov flow with [Julia](https://julialang.org/) using a ["Stable Fluids"](https://www.josstam.com/_files/ugd/cf1fd6_898fe9b63df946689101b8d074f8efba.pdf) algorithm following this video: [Writing a Turbulence Simulation in Julia](https://www.youtube.com/watch?v=26vdFV1EXKk). Note that the Stable Fluids algorithm is **not** as accurate as sophisticated computational fluid dynamics (CFD) methods, because it was proposed for real-time applications in computer graphics, as stated in the original paper.

## Stable Fluids algorithm

Our goal is to find the next velocity $\mathbf{u}(\mathbf{x}, t+\Delta t)$ from the current velocity $\mathbf{u}(\mathbf{x}, t)$ given that other variables are known.

The basic idea of the Stable Fluids algorithm is to break down the Navier-Stokes equations into series of steps.

1. Add Force

$$
\frac{\partial \mathbf{u}}{\partial t} = \mathbf{f}
$$

2. Advect

$$
\frac{\partial \mathbf{u}}{\partial t} + \mathbf{u} \cdot \nabla \mathbf{u} = 0
$$

3. Diffuse

$$
\frac{\partial \mathbf{u}}{\partial t} = \nu \nabla^2 \mathbf{u}
$$

4. Project

$$
\frac{\partial \mathbf{u}}{\partial t} = -\frac{1}{\rho}\nabla p
$$

$$
\nabla\cdot \mathbf{u} = 0
$$

We approximate $\frac{\partial \mathbf{u}}{\partial t}$ by a forward difference:

$$
\frac{\partial \mathbf{u}}{\partial t} \approx \frac{\mathbf{u}(\mathbf{x}, t + \Delta t) - \mathbf{u}(\mathbf{x}, t)}{\Delta t}
$$

Therefore, the basic algorithm is as follows:

At current time $t$,

0. Initialize 

$$
\mathbf{w}_0(\mathbf{x}) = \mathbf{u}(\mathbf{x}, t)
$$

1. Add Force

$$
\frac{\mathbf{w}_1(\mathbf{x}) - \mathbf{w}_0(\mathbf{x})}{\Delta t} = \mathbf{f}(\mathbf{x}, t)
$$

$$
\rightarrow \mathbf{w}_1(\mathbf{x}) = \mathbf{w}_0(\mathbf{x}) + \Delta t \; \mathbf{f}(\mathbf{x}, t)
$$

2. Advect

$$
\mathbf{w}_2(\mathbf{x}) = \mathbf{w}_1(\mathbf{p}(\mathbf{x},-\Delta t))
$$

where $\mathbf{p}(\mathbf{x},-\Delta t)$ is the position at time $t - \Delta t$ of the particle that moves to $\mathbf{x}$ at time $t$ according to the path $\mathbf{p}$ given by the velocity field $\mathbf{w}_0(\mathbf{x})$.

We use a simple method to find the "backtraced position" $\mathbf{p}(\mathbf{x},-\Delta t)$ using the definition of the velocity (i.e., the time derivative of the position):

$$
\frac{\mathbf{x} - \mathbf{p}(\mathbf{x},-\Delta t)}{\Delta t} = \mathbf{w}_1(\mathbf{x})
$$

$$
\rightarrow \mathbf{p}(\mathbf{x},-\Delta t) = \mathbf{x} - \Delta t \; \mathbf{w}_1(\mathbf{x})
$$

3. Diffuse

We use an implicit method to solve the diffusion equation:

$$
\frac{\mathbf{u}(\mathbf{x}, t + \Delta t) - \mathbf{u}(\mathbf{x}, t)}{\Delta t} = \nu \nabla^2 \mathbf{u}(\mathbf{x}, t + \Delta t)
$$

$$
\rightarrow \frac{\mathbf{w}_3(\mathbf{x}) - \mathbf{w}_2(\mathbf{x})}{\Delta t} = \nu \nabla^2 \mathbf{w}_3(\mathbf{x})
$$

$$
\rightarrow \mathbf{w}_3(\mathbf{x}) - \nu \Delta t \nabla^2 \mathbf{w}_3(\mathbf{x}) = \mathbf{w}_2(\mathbf{x})
$$

$$
\rightarrow (\mathrm{I} - \nu \Delta t \nabla^2) \mathbf{w}_3(\mathbf{x}) = \mathbf{w}_2(\mathbf{x})
$$

where $\mathrm{I}$ is the identity operator. When we discretize the Laplace operator $\nabla^2$, this becomes a sparse linear system for the unknown $\mathbf{w}_3(\mathbf{x})$.

4. Project

The pressure equation can be discretized as follows:

$$
\frac{\mathbf{u}(\mathbf{x}, t + \Delta t) - \mathbf{u}(\mathbf{x}, t)}{\Delta t} = -\frac{1}{\rho}\nabla p(\mathbf{x})
$$

We use the pressure equation so that the velocity $\mathbf{w}_4(\mathbf{x})$ is divergence-free.

$$
\frac{\mathbf{w}_4(\mathbf{x}) - \mathbf{w}_3(\mathbf{x})}{\Delta t} = -\frac{1}{\rho}\nabla p(\mathbf{x}) \qquad \text{s.t.} \quad \nabla\cdot \mathbf{w}_4(\mathbf{x}) = 0
$$

$$
\rightarrow \mathbf{w}_4(\mathbf{x}) = \mathbf{w}_3(\mathbf{x}) -  \frac{\Delta t}{\rho} \; \nabla p(\mathbf{x})
$$

$$
\rightarrow \nabla\cdot \mathbf{w}_4(\mathbf{x}) = \nabla \cdot \mathbf{w}_3(\mathbf{x}) - \frac{\Delta t}{\rho} \; \nabla^2 p(\mathbf{x})
$$

$$
\rightarrow 0 = \nabla \cdot \mathbf{w}_3(\mathbf{x}) - \frac{\Delta t}{\rho} \; \nabla^2 p(\mathbf{x})
$$

$$
\rightarrow \nabla^2 p(\mathbf{x}) = \frac{\rho}{\Delta t} \; \nabla \cdot \mathbf{w}_3(\mathbf{x})
$$

Since $\mathbf{w}_3(\mathbf{x})$ is known, this is a Poisson equation for the pressure $p(\mathbf{x})$. After solving the Poisson equation, we can compute the final velocity $\mathbf{w}_4(\mathbf{x})$. That is,

$$
\mathbf{w}_4(\mathbf{x}) = \mathbf{w}_3(\mathbf{x}) -  \frac{\Delta t}{\rho} \; \nabla p(\mathbf{x})
$$

Here, $p(\mathbf{x})$ is given by solving the Poisson equation:

$$
\nabla^2 p(\mathbf{x}) = \frac{\rho}{\Delta t} \; \nabla \cdot \mathbf{w}_3(\mathbf{x})
$$

Finally, $\mathbf{w}_4(\mathbf{x})$ is the next velocity $\mathbf{u}(\mathbf{x}, t + \Delta t)$.

$$
\mathbf{u}(\mathbf{x}, t + \Delta t) = \mathbf{w}_4(\mathbf{x})
$$

### Summary

#### 0. Initialize

$$
\mathbf{w}_0(\mathbf{x}) = \mathbf{u}(\mathbf{x}, t)
$$

#### 1. Add Force

$$
\mathbf{w}_1(\mathbf{x}) = \mathbf{w}_0(\mathbf{x}) + \Delta t \; \mathbf{f}(\mathbf{x}, t)
$$

#### 2. Advect

$$
\mathbf{w}_2(\mathbf{x}) = \mathbf{w}_1(\mathbf{x} - \Delta t \; \mathbf{w}_1(\mathbf{x}))
$$

#### 3. Diffuse

$$
(\mathrm{I} - \nu \Delta t \nabla^2) \mathbf{w}_3(\mathbf{x}) = \mathbf{w}_2(\mathbf{x})
$$

#### 4. Project

##### 4-1. Solve the Poisson equation

$$
\nabla^2 p(\mathbf{x}) = \frac{\rho}{\Delta t} \; \nabla \cdot \mathbf{w}_3(\mathbf{x})
$$

##### 4-2. Compute the next velocity

$$
\mathbf{u}(\mathbf{x}, t + \Delta t) = \mathbf{w}_3(\mathbf{x}) -  \frac{\Delta t}{\rho} \; \nabla p(\mathbf{x})
$$

## Stable Fluids algorithm using FFT

When we consider a domain with periodic boundary conditions, we can use the Fourier transform to solve the diffusion equation and the Poisson equation efficiently. Numerically, we can use the Fast Fourier Transform (FFT) for this purpose.

In [2]:
# Julia test
print("Hello world")

Hello world

```{bibliography}
```